In [42]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [43]:
# Load your data
btc_data = pd.read_csv('csv\\BTCUSD_1m_btc_usd_60.csv')
eth_data = pd.read_csv('csv\\ETHUSD_1m_eth_usd_60.csv')
ltc_data = pd.read_csv('csv\\LTCUSD_1m_ltc_usd_60.csv')
xrp_data = pd.read_csv('csv\\XRPUSD_1m_xrp_usd_60.csv')

# Convert the 'time' column to datetime and set as index
btc_data['time'] = pd.to_datetime(btc_data['time'])
eth_data['time'] = pd.to_datetime(eth_data['time'])
ltc_data['time'] = pd.to_datetime(ltc_data['time'])
xrp_data['time'] = pd.to_datetime(xrp_data['time'])

btc_data.set_index('time', inplace=True)
eth_data.set_index('time', inplace=True)
ltc_data.set_index('time', inplace=True)
xrp_data.set_index('time', inplace=True)

# Training period
start_date = '2021-09-14 00:00:00'
end_date = '2023-04-19 00:00:00'

# Filter data for the training period
btc_train = btc_data.loc[start_date:end_date]
eth_train = eth_data.loc[start_date:end_date]
ltc_train = ltc_data.loc[start_date:end_date]
xrp_train = xrp_data.loc[start_date:end_date]


In [12]:
# Calculate returns for the training period
btc_returns_train = btc_train['Close'].pct_change().dropna()
eth_returns_train = eth_train['Close'].pct_change().dropna()
ltc_returns_train = ltc_train['Close'].pct_change().dropna()
xrp_returns_train = xrp_train['Close'].pct_change().dropna()

# Combine returns into a single DataFrame
train_returns = pd.concat([btc_returns_train, eth_returns_train, ltc_returns_train, xrp_returns_train], axis=1)
train_returns.columns = ['BTC', 'ETH', 'LTC', 'XRP']

# Calculate the historical volatility (standard deviation) of each asset
volatility = train_returns.std()

# Define the objective function for risk parity
def risk_parity_objective(weights, volatilities):
    # Calculate portfolio variance
    portfolio_variance = np.dot(weights ** 2, volatilities ** 2)

    # Calculate marginal risk contribution of each asset
    marginal_risk_contribution = weights * volatilities ** 2 / np.sqrt(portfolio_variance)

    # Calculate risk contributions and target risk contribution (equal for all assets)
    risk_contributions = marginal_risk_contribution / marginal_risk_contribution.sum()
    target_risk_contribution = np.ones(len(weights)) / len(weights)

    # Objective function: minimize the sum of squared differences between actual and target risk contributions
    return np.sum((risk_contributions - target_risk_contribution) ** 2)

# Constraints and bounds for optimization
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(train_returns.shape[1]))

# Initial guess (equal weighting)
init_guess = [1.0 / train_returns.shape[1]] * train_returns.shape[1]

# Optimize portfolio for risk parity
optimal_weights_risk_parity = minimize(risk_parity_objective, init_guess, args=(volatility,), method='SLSQP', bounds=bounds, constraints=constraints)

print("Optimal Weights:", optimal_weights_risk_parity.x)



Optimal Weights: [0.50477571 0.3363203  0.1448375  0.01406649]


In [13]:
# Separate data for backtesting (data outside the training period)
btc_test = btc_data.loc[end_date:]
eth_test = eth_data.loc[end_date:]
ltc_test = ltc_data.loc[end_date:]
xrp_test = xrp_data.loc[end_date:]

# Calculate returns for the test period
btc_returns_test = btc_test['Close'].pct_change().dropna()
eth_returns_test = eth_test['Close'].pct_change().dropna()
ltc_returns_test = ltc_test['Close'].pct_change().dropna()
xrp_returns_test = xrp_test['Close'].pct_change().dropna()

# Combine test returns into a single DataFrame
test_returns = pd.concat([btc_returns_test, eth_returns_test, ltc_returns_test, xrp_returns_test], axis=1)
test_returns.columns = ['BTC', 'ETH', 'LTC', 'XRP']

# Ensure optimal weights are in a numpy array for calculations
optimal_weights_array = np.array(optimal_weights_risk_parity.x)

# Calculate the portfolio returns for the test period
test_portfolio_returns = test_returns.dot(optimal_weights_array)

# You might want to analyze these returns further, e.g., compute total return, volatility, etc.
total_return = np.prod(1 + test_portfolio_returns) - 1
volatility = test_portfolio_returns.std()

print("Total Return:", total_return)
print("Volatility:", volatility)



Total Return: -0.08497749939135113
Volatility: 0.0006319397963427223


In [40]:
100000*optimal_weights_risk_parity.x

array([50477.57108862, 33632.03013148, 14483.7496752 ,  1406.64910471])

In [41]:
sum(optimal_weights_risk_parity.x)

1.0